In [8]:
import pandas as pd
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
import numpy as np
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt

In [9]:
from IPython.display import display, Markdown
def display_df(df):
    display(Markdown(df.to_markdown()))

In [10]:
infochamigo = pd.read_excel('infochamigo.xlsx')
infochamigo['ventaid'] = infochamigo['ventaid'].dropna()
infochamigo['maestroid'] = infochamigo['maestroid'].dropna()
infochamigo.head()

,nroticket,fechahora,cantidad,preciounitario,importebruto,descuento,importeneto,ruc,cliente,unidadmedida,...,segmentopdv,marcamaestro,codsucursal,sucursal,idpdv,rango_horario,semana_anho,categoria,ventaid,maestroid
0,1,2021-11-23 17:34:28.201,8,2.500000e+19,20000.0,0,20000.0,0,SIN NOMBRE,NaN,...,NaN,KAISER ULTRA CERO,1,1000077 - BODEGA CHIMI CENTRAL,252437,4 AFTER,47.0,CERVEZAS,7508,CERVEZAS49
1,1,2021-11-25 16:44:25.367,1,5.000000e+19,5000.0,0,5000.0,0,SIN NOMBRE,NaN,...,NaN,KRO,1,Minimarket Duarte CENTRAL,252723,3 SIESTA,47.0,BOCADITO DE MAIZ,8212,GRAL9074
2,1,2021-11-30 16:08:18.110,1,3.000000e+19,3000.0,0,3000.0,0,SIN NOMBRE,NaN,...,Alto1,ARCOR,1,37239-KIOSKO PAMELA CENTRAL,227210,3 SIESTA,48.0,NaN,10355,GRAL8881
3,1,2021-11-30 17:11:37.508,12,3.333333e+19,40000.0,0,40000.0,0,SIN NOMBRE,NaN,...,Alto1,SKOL,1,405877-PARIS BEBIDAS CENTRAL,77323,4 AFTER,48.0,CERVEZAS,10375,CERVEZAS96
4,1,2021-11-30 17:11:37.508,60,3.333333e+19,200000.0,0,200000.0,0,SIN NOMBRE,NaN,...,Alto1,SKOL CONTRABANDO,1,405877-PARIS BEBIDAS CENTRAL,77323,4 AFTER,48.0,CERVEZAS,10375,CERVEZAS1730


In [11]:
infochamigo['maestroid'].dropna()

0          CERVEZAS49
1            GRAL9074
2            GRAL8881
3          CERVEZAS96
4        CERVEZAS1730
             ...     
59994       GRAL11876
59996        GRAL4970
59997        GRAL3733
59998        GRAL5978
59999        GRAL7469
Name: maestroid, Length: 48332, dtype: object

In [12]:
piv_table = pd.crosstab(index = infochamigo['ventaid'], columns = infochamigo['maestroid'])
piv_table.to_csv('compras_table.csv', index = True)
display(piv_table.head())

maestroid,CERVEZAS10,CERVEZAS100,CERVEZAS101,CERVEZAS102,CERVEZAS103,CERVEZAS104,CERVEZAS105,CERVEZAS11,CERVEZAS110,CERVEZAS111,...,GRAL9788,GRAL98,GRAL9803,GRAL9818,GRAL9847,GRAL9849,GRAL9852,GRAL99,GRAL990,GRAL9973
ventaid,,,,,,,,,,,,,,,,,,,,,
317,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
326,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
330,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
331,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
339,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# ------------------------- Detect how many clusters to use ------------------------

# ks = range(1, 200, 10)
# inertias = []

# for k in ks:
#     # Create a KMeans instance with k clusters: model
#     model = KMeans(n_clusters=k)
    
#     # Fit model to samples
#     model.fit(piv_table)
    
#     # Append the inertia to the list of inertias
#     inertias.append(model.inertia_)

# plt.rcParams["figure.dpi"] = 200
# # Plot ks vs inertias
# plt.plot(ks, inertias, '-o')
# plt.xlabel('number of clusters, k')
# plt.xticks(rotation = 45, size = 5)
# plt.ylabel('inertia')
# plt.xticks(ks)
# plt.show()


In [14]:
# Create a KMeans instance: model
model = KMeans(n_clusters=200)

# Fit model to points
model.fit(piv_table)
import pickle
pickle.dump(model, open('pickles/recomendaciones_compras_200_clusters.pickle', 'wb')) #Saving the model

# Determine the cluster labels of new_points: labels
labels = model.predict(piv_table)

df = pd.DataFrame({'labels': labels, 'compra': piv_table.index}).sort_values('labels')

# Display df sorted by cluster label
display(df.sort_values('labels', ascending=False).head())

,labels,compra
21759,199,67079
23315,199,70509
7134,199,22915
17374,199,56330
23526,199,70903


# Normalizer: {1,2,3,4,5} --> {1.,1.,1.,1.,1.}

In [15]:
# Create a KMeans model with n clusters: kmeans
kmeans = KMeans(n_clusters=200)

# Create a normalizer: normalizer
normalizer = Normalizer()

# Make a pipeline chaining normalizer and kmeans: pipeline
pipeline_with_normalizer = make_pipeline(normalizer, kmeans)

# Fit pipeline to the daily price movements
pipeline_with_normalizer.fit(piv_table)
pickle.dump(pipeline_with_normalizer, open('pickles/recomendaciones_compras_normalizer_200_clusters.pickle', 'wb')) #Saving the model
# Predict the cluster labels: labels
labels = pipeline_with_normalizer.predict(piv_table)

# Create a DataFrame aligning labels and compras
normalized_labels = pd.DataFrame({'labels': labels, 'compra': piv_table.index}).sort_values('labels')

# Create crosstab: ct
normalized_ct = pd.crosstab(normalized_labels['labels'], normalized_labels['compra'])

# Display df sorted by cluster label
display(normalized_labels.sort_values('labels', ascending=False))



,labels,compra
10367,199,33699
12261,199,40309
10779,199,35761
13558,199,45027
3872,199,14136
...,...,...
6454,0,21067
4393,0,15468
20040,0,62616
23685,0,71197


# StandardScaler: {1,2,3,4,5} --> {−1.41,−0.71,0.,0.71,1.41}

In [16]:
# # Create scaler: scaler
scaler = StandardScaler()

# Create pipeline with StandardScaler
pipeline_with_standard_scaler = make_pipeline(scaler, kmeans)

# Fit pipeline to the daily price movements
pipeline_with_standard_scaler.fit(piv_table)
pickle.dump(pipeline_with_standard_scaler, open('pickles/recomendaciones_compras_standard_scaler_200_clusters.pickle', 'wb'))

# Predict the cluster labels: labels
labels = pipeline_with_standard_scaler.predict(piv_table)

# Create a DataFrame aligning labels and symbols: symbols_labels
std_scaler_labels = pd.DataFrame({'labels': labels, 'compra': piv_table.index}).sort_values('labels')

# Create crosstab: ct
std_scaler_ct = pd.crosstab(std_scaler_labels['labels'], std_scaler_labels['compra'])

# Display df sorted by cluster label
display(std_scaler_labels.sort_values('labels', ascending=False))


,labels,compra
16318,199,53178
2099,198,3428
18406,197,58751
10031,196,32100
13164,195,43790
...,...,...
20774,0,64685
495,0,1539
499,0,1543
21461,0,66384
